# Finding Tiny Faces in the Wild with Generative Adversarial Network
implementation in **keras with tensorflow backend**.

Link to paper: https://ivul.kaust.edu.sa/Pages/pub-tiny-faces.aspx

# Dataset download (Cropped Thumbnails of faces)

In [ ]:
# Code to Load Regions of Interest (ROI) i.e. Faces and Non Faces. Link: https://ivul.kaust.edu.sa/Pages/pub-tiny-faces.aspx

from __future__ import print_function
import os
import sys
import gzip
import json
import shutil
import zipfile
import requests
import subprocess
from tqdm import tqdm
from six.moves import urllib

def download_file_from_google_drive(fileid, path):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params={'id': fileid}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {'id': fileid, 'confirm': token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, path)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None


def save_response_content(response, path):
    CHUNK_SIZE = 32768

    with open(path, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)

def download(url, dirpath):
  filename = url.split('/')[-1]
  filepath = os.path.join(dirpath, filename)
  u = urllib.request.urlopen(url)
  f = open(filepath, 'wb')
  filesize = int(u.headers["Content-Length"])
  print("Downloading: %s Bytes: %s" % (filename, filesize))

  downloaded = 0
  block_sz = 8192
  status_width = 70
  while True:
    buf = u.read(block_sz)
    if not buf:
      print('')
      break
    else:
      print('', end='\r')
    downloaded += len(buf)
    f.write(buf)
    status = (("[%-" + str(status_width + 1) + "s] %3.2f%%") %
      ('=' * int(float(downloaded) / filesize * status_width) + '>', (downloaded * 100. / filesize * 8192)))
    print(status, end='')
    sys.stdout.flush()
  f.close()
  return filepath

def download_file_from_google_drive(id, destination):
  URL = "https://docs.google.com/uc?export=download"
  session = requests.Session()

  response = session.get(URL, params={ 'id': id }, stream=True)
  token = get_confirm_token(response)

  if token:
    params = { 'id' : id, 'confirm' : token }
    response = session.get(URL, params=params, stream=True)

  save_response_content(response, destination)

def get_confirm_token(response):
  for key, value in response.cookies.items():
    if key.startswith('download_warning'):
      return value
  return None

def save_response_content(response, destination, chunk_size=32*1024):
  total_size = int(response.headers.get('content-length', 0))
  with open(destination, "wb") as f:
    for chunk in tqdm(response.iter_content(chunk_size), total=total_size,
              unit='B', unit_scale=True, desc=destination):
      if chunk: # filter out keep-alive new chunks
        f.write(chunk)

def unzip(filepath):
  print("Extracting: " + filepath)
  dirpath = os.path.dirname(filepath)
  with zipfile.ZipFile(filepath) as zf:
    zf.extractall(dirpath)
  os.remove(filepath)

def download_file(dirpath, filename, drive_id):
  data_dir = 'ROI'
#   if os.path.exists(os.path.join(dirpath, data_dir)):
#     print('Found ROI - skip')
#     return

  #filename, drive_id  = "WIDER_train.zip", "0B6eKvaijfFUDQUUwd21EckhUbWs"
  save_path = os.path.join(dirpath, filename)

#   if os.path.exists(save_path):
#     print('[*] {} already exists'.format(save_path))
#   else:
  download_file_from_google_drive(drive_id, save_path)

  zip_dir = ''
  with zipfile.ZipFile(save_path) as zf:
    zip_dir = zf.namelist()[0]
    zf.extractall(dirpath)
  os.remove(save_path)
  os.rename(os.path.join(dirpath, zip_dir), os.path.join(dirpath, data_dir))

if __name__ == '__main__':
  download_file('/', 'nthumbs.zip' ,'1r8rY_1f76yzNdYz9RKwOw5AhIXdma0kp')
  download_file('/', 'thumbs.zip' ,'1XbkaHeY1sg5vYVn1nj1qThHH9xSG33mb')
  download_file('/', 'LRthumbs.zip' ,'1yuCwXoVHCBx0A_TCNER696XMzAutHx-9')
  download_file('/', 'LRnthumbs.zip' ,'1IFcxjsnG_aRNB8PLYjbRcDv53ivnc8vr')
  
nremove = !ls nthumbs | head -1
remove = !ls thumbs | head -1
!rm /content/thumbs/{remove[0]}
!rm /content/nthumbs/{nremove[0]}

/nthumbs.zip: 784B [00:00, 2.10kB/s]
/thumbs.zip: 1.09kB [00:00, 2.34kB/s]
/LRthumbs.zip: 491B [00:00, 3.31kB/s]
/LRnthumbs.zip: 450B [00:00, 2.90kB/s]


In [ ]:
import glob
import numpy as np
import cv2
from PIL import Image
fileListThumbs = glob.glob('thumbs/*.jpg')
fileListNotthumbs = glob.glob('nthumbs/*.jpg')
LRfileListThumbs = glob.glob('LRthumbs/*.jpg')
LRfileListNotthumbs = glob.glob('LRnthumbs/*.jpg')
thumbs = np.array([np.array(Image.open(fname)) for fname in fileListThumbs]) #All thumbs (18298) as numpy array
notThumbs = np.array([np.array(Image.open(fname)) for fname in fileListNotthumbs])
LRthumbs = np.array([np.array(Image.open(fname)) for fname in LRfileListThumbs]) #All LR thumbs (18298) as numpy array
LRnotThumbs = np.array([np.array(Image.open(fname)) for fname in LRfileListNotthumbs])

# Model Preparation and Training 


In [ ]:
def normalization(X):
    return X / 127.5 - 1  #To Bring pixel values in range [-1, 1]

def gen_batch(X, batch_size):
  #X is numpy array of all files
    while True:
        idx = np.random.choice(X.shape[0], batch_size, replace=False) #Generates a random batch from the dataset
        yield X[idx] #Return files with yield on the go

In [ ]:
from __future__ import print_function, division
import scipy


from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add, MaxPooling2D, Flatten
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
import keras.backend as K


import sys
import os
import datetime


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline



Using TensorFlow backend.


In [ ]:
from keras.applications.vgg19 import VGG19

In [ ]:
channels=3
n_residual_blocks = 8

In [ ]:
lr_shape=(12,12,channels)
hr_shape=(48,48,channels)

In [ ]:
alpha = 0.001
beta = 0.01

In [ ]:
def residual_block(layer_input, filters):
  d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(layer_input)
  d = Activation('relu')(d)
  d = BatchNormalization(momentum=0.9)(d)
  d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
  d = Activation('relu')(d)
  d = BatchNormalization(momentum=0.9)(d)
  d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
  d = Activation('relu')(d)
  d = BatchNormalization(momentum=0.9)(d)
  d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
  d = Activation('relu')(d)
  d = BatchNormalization(momentum=0.9)(d)
  d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
  d = Activation('relu')(d)
  d = BatchNormalization(momentum=0.9)(d)
  d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
  d = Activation('relu')(d)
  d = BatchNormalization(momentum=0.9)(d)
  d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
  d = Activation('relu')(d)
  d = BatchNormalization(momentum=0.9)(d)
  d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
  d = BatchNormalization(momentum=0.9)(d)
  d = Add()([d, layer_input])
  return d



In [ ]:
def deconv2d(layer_input):
  u = UpSampling2D(size=2)(layer_input)
  u = Conv2D(256, kernel_size=3, strides=1, padding='same')(u)
  u = Activation('relu')(u)
  return u
  

In [ ]:
img_lr = Input(shape=lr_shape)
  
  # Pre-residual block
cpr1 = Conv2D(64, kernel_size=9, strides=1, padding='same')(img_lr)
cpr1 = Activation('relu')(cpr1)

  # Propogate through residual blocks
r1 = residual_block(cpr1,64)
for _ in range(n_residual_blocks - 1):
  r1 = residual_block(r1, 64)

  # Post-residual block
cpr2 = Conv2D(64, kernel_size=3, strides=1, padding='same')(r1)
cpr2 = BatchNormalization(momentum=0.9)(cpr2)
cpr2 = Add()([cpr2, cpr1])

  # Upsampling
u1 = deconv2d(cpr2)
u2 = deconv2d(u1)

  
inter_sr=Conv2D(channels, kernel_size=1, strides=1, padding='same')(u2)
  
##refinement network
  
  # Pre-residual block
cpr3 = Conv2D(64, kernel_size=9, strides=1, padding='same')(inter_sr)
cpr3 = Activation('relu')(cpr3)

  # Propogate through residual blocks
r2 = residual_block(cpr3,64)
for _ in range(n_residual_blocks - 1):
  r2 = residual_block(r2, 64)

  # Post-residual block
cpr4 = Conv2D(64, kernel_size=3, strides=1, padding='same')(r2)
cpr4 = BatchNormalization(momentum=0.9)(cpr4)
cpr5 = Conv2D(256, kernel_size=3, strides=1, padding='same')(cpr4)
cpr5 = BatchNormalization(momentum=0.9)(cpr5)
cpr6 = Conv2D(256, kernel_size=3, strides=1, padding='same')(cpr5)
cpr6 = BatchNormalization(momentum=0.9)(cpr6)

  
img_sr=Conv2D(channels, kernel_size=3, strides=1, padding='same')(cpr6)

generator=Model(img_lr, [inter_sr, img_sr])


In [ ]:
generator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12, 12, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 12, 12, 64)   15616       input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 12, 12, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 12, 12, 64)   36928       activation_1[0][0]               
__________________________________________________________________________________________________
activation

### We employ VGG19 as our backbone network in the discriminator

In [ ]:
vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(48,48,3))

80142336/80134624 [==============================] - 15s 0us/step


In [ ]:
vgg19.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [ ]:
vgg19.layers

In [ ]:
X = Flatten()(vgg19.layers[-2].output)
Fc_RorG=Dense(1, activation='sigmoid')(X)  ###check for real vs. generated image
Fc_ForNF=Dense(1,activation='sigmoid')(X)  ###check for face vs. non-face

trail_discriminator=Model(inputs = vgg19.input, outputs = [Fc_RorG,Fc_ForNF])
#### there are two outputs for the discriminator!!

In [ ]:
trail_discriminator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 48, 48, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 48, 48, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 48, 48, 64)   36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 24, 24, 64)   0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

### When we apply binary_crossentropy to both the parallel outputs of discriminator we attempt at maximizing the adversarial loss and minimizing the classification loss.....

In [ ]:
trail_discriminator.compile(optimizer=Adam(lr=1e-3), loss=['binary_crossentropy', 'binary_crossentropy'], loss_weights=[alpha, beta])
trail_discriminator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 48, 48, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 48, 48, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 48, 48, 64)   36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 24, 24, 64)   0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

### We will create model with generator and discriminator stacked to train the generator!!

In [ ]:
# High res. and low res. images
img_hr = Input(shape=hr_shape)
img_lr = Input(shape=lr_shape)

# Generate super resolution version from low resolution version of an image.
inter_sr, img_sr = generator(img_lr) #super-resolution : G1(ILR) , #refinement : G2(G1(ILR))
validity, face = trail_discriminator(img_sr)

GD_combined = Model([img_lr, img_hr], [validity, face, inter_sr, img_sr])
### there are 4 outputs from complete GAN model: 'validity' for adversarial loss, 'face' for classification loss, 'inter_sr' and 'img_sr' for pixel-wise loss.
### All these losses will be minimized to train the generator!!!

### Before compiling the combine model we need to freeze the discriminator weights!!

In [ ]:
trail_discriminator.trainable = False

In [ ]:
GD_combined.compile(optimizer=Adam(lr=1e-3), loss=['binary_crossentropy', 'binary_crossentropy', 'mse', 'mse'],loss_weights=[alpha, beta, 1, 1])
GD_combined.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 12, 12, 3)         0         
_________________________________________________________________
model_1 (Model)              [(None, 48, 48, 3), (None 6350470   
_________________________________________________________________
model_3 (Model)              [(None, 1), (None, 1)]    20033602  
Total params: 26,384,072
Trainable params: 6,332,806
Non-trainable params: 20,051,266
_________________________________________________________________


### the definition of train function is incomplete since our input images batch is not ready!!!
### But the model.train_on_batch function is ready for training discriminator and generator!!

In [ ]:
def train(epochs, batch_size=1):
  start_time = datetime.datetime.now()

  for epoch in range(epochs):
   
    # ----------------------
    #  Train Discriminator
    # ----------------------
    
    # Sample images and their conditioning counterparts
    # NOTE: how will we load the batch of data is yet to figure out. So this line is just written for represention of that task!!
    imgs_hr, imgs_lr, y = load_data(batch_size) ##################IMPORTANT TO FEED#########################
    
    # From low res. image generate high res. version
    inter_sr, img_sr = generator.predict(imgs_lr)
    
    valid = np.ones((batch_size,))
    fake = np.zeros((batch_size,))
    
    d_loss_real = trail_discriminator.train_on_batch(imgs_hr, [valid,y]) ### there are two outputs for discriminator and training will take place taking into account of both of them
    d_loss_fake = trail_discriminator.train_on_batch(img_sr, [fake,y])
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ------------------
    #  Train Generator
    # ------------------

    # Sample images and their conditioning counterparts
    imgs_hr, imgs_lr, y  = load_data(batch_size) #######################IMPORTANT TO FEED#####################

    # The generators want the discriminators to label the generated images as real
    valid = np.ones((batch_size,))

    # Train the generators
    g_loss = GD_combined.train_on_batch([imgs_lr, imgs_hr], [valid, y, imgs_hr, img_hr])    
    
    
    elapsed_time = datetime.datetime.now() - start_time
    # Plot the progress
    print ("%d time: %s" % (epoch, elapsed_time))

In [ ]:
####start training

train(epochs,batch_size)